In [1]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as cp

# ase imports
import ase.io
from ase import Atoms, Atom
from ase import units
from ase.build import molecule
# for MD
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory

In [2]:
# helper functions
def make_water(density, super_cell=[3, 3, 3]):
    """ Geenrates a supercell of water molecules with a desired density.
        Density in g/cm^3!!!"""
    h2o = molecule('H2O')
    a = np.cbrt((sum(h2o.get_masses()) * units.m ** 3 * 1E-6 ) / (density * units.mol))
    h2o.set_cell((a, a, a))
    h2o.set_pbc((True, True, True))
    #return cp(h2o.repeat(super_cell))
    return h2o.repeat(super_cell)

def rms_dict(x_ref, x_pred):
    """ Takes two datasets of the same shape and returns a dictionary containing RMS error data"""

    x_ref = np.array(x_ref)
    x_pred = np.array(x_pred)

    if np.shape(x_pred) != np.shape(x_ref):
        raise ValueError('WARNING: not matching shapes in rms')

    error_2 = (x_ref - x_pred) ** 2

    average = np.sqrt(np.average(error_2))
    std_ = np.sqrt(np.var(error_2))

    return {'rmse': average, 'std': std_}

In [3]:
# Running MD with ASE's EMT

from ase.calculators.emt import EMT
calc = EMT()

T = 150  # Kelvin

# Set up a grid of water
water = make_water(1.0, [3, 3, 3])
water.set_calculator(calc)

# We want to run MD using the Langevin algorithm
# with a time step of 1 fs, the temperature T and the friction
# coefficient to 0.002 atomic units.
dyn = Langevin(water, 1 * units.fs, T * units.kB, 0.0002)

def printenergy(a=water):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

dyn.attach(printenergy, interval=5)

# We also want to save the positions of all atoms after every 5th time step.
traj = Trajectory('dyn_emt.traj', 'w', water)
dyn.attach(traj.write, interval=5)

# Now run the dynamics
printenergy(water)
dyn.run(600)   # CHANGE THIS IF YOU WANT LONGER/SHORTER RUN



/Users/gsivaraman/Documents/projects/DSL/ML_IP_2021/conda_psik/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Energy per atom: Epot = 0.885eV  Ekin = 0.000eV (T=  0K)  Etot = 0.885eV
Energy per atom: Epot = 0.885eV  Ekin = 0.000eV (T=  0K)  Etot = 0.885eV
Energy per atom: Epot = 0.820eV  Ekin = 0.053eV (T=413K)  Etot = 0.874eV
Energy per atom: Epot = 0.660eV  Ekin = 0.208eV (T=1610K)  Etot = 0.868eV
Energy per atom: Epot = 0.632eV  Ekin = 0.224eV (T=1734K)  Etot = 0.857eV
Energy per atom: Epot = 0.869eV  Ekin = 0.005eV (T= 39K)  Etot = 0.874eV
Energy per atom: Epot = 0.781eV  Ekin = 0.089eV (T=685K)  Etot = 0.869eV
Energy per atom: Epot = 0.755eV  Ekin = 0.117eV (T=906K)  Etot = 0.873eV
Energy per atom: Epot = 0.707eV  Ekin = 0.163eV (T=1263K)  Etot = 0.870eV
Energy per atom: Epot = 0.703eV  Ekin = 0.159eV (T=1230K)  Etot = 0.862eV
Energy per atom: Epot = 0.865eV  Ekin = 0.009eV (T= 71K)  Etot = 0.874eV
Energy per atom: Epot = 0.732eV  Ekin = 0.136eV (T=1049K)  Etot = 0.868eV
Energy per atom: Epot = 0.692eV  Ekin = 0.177eV (T=1370K)  Etot = 0.869eV
Energy per atom: Epot = 0.826eV  Ekin = 0.046

Energy per atom: Epot = 0.419eV  Ekin = 0.618eV (T=4782K)  Etot = 1.037eV
Energy per atom: Epot = 0.412eV  Ekin = 0.625eV (T=4833K)  Etot = 1.037eV
Energy per atom: Epot = 0.396eV  Ekin = 0.644eV (T=4985K)  Etot = 1.041eV
Energy per atom: Epot = 0.389eV  Ekin = 0.638eV (T=4935K)  Etot = 1.027eV
Energy per atom: Epot = 0.406eV  Ekin = 0.621eV (T=4802K)  Etot = 1.026eV
Energy per atom: Epot = 0.355eV  Ekin = 0.674eV (T=5215K)  Etot = 1.029eV
Energy per atom: Epot = 0.458eV  Ekin = 0.572eV (T=4428K)  Etot = 1.030eV
Energy per atom: Epot = 0.466eV  Ekin = 0.565eV (T=4370K)  Etot = 1.031eV
Energy per atom: Epot = 0.405eV  Ekin = 0.618eV (T=4781K)  Etot = 1.023eV
Energy per atom: Epot = 0.448eV  Ekin = 0.786eV (T=6078K)  Etot = 1.234eV
Energy per atom: Epot = 0.434eV  Ekin = 0.966eV (T=7471K)  Etot = 1.400eV


True

In [4]:
# wrap and save traj in .xyz --- the .traj is a non human readable database file, xyz is much better
out_traj = ase.io.read('dyn_emt.traj', ':')
for at in out_traj:
    at.wrap()
    if 'momenta' in at.arrays: del at.arrays['momenta']
ase.io.write('dyn_emt.xyz', out_traj, 'xyz')


In [9]:
from dpdata import MultiSystems

ase_multi_system = MultiSystems.from_file(file_name='dyn_emt.traj',fmt='ase/structure')

In [10]:
print(ase_multi_system)
print(ase_multi_system.systems) # return a dictionaries

MultiSystems (1 systems containing 121 frames)
{'H54O27': Data Summary
Labeled System
-------------------
Frame Numbers      : 121
Atom Numbers       : 81
Including Virials  : No
Element List       :
-------------------
H  O
54  27}


In [11]:
# print the system infomation
print(ase_multi_system.systems['H54O27'].data)


# dump all systems
ase_multi_system.to_deepmd_raw('./my_deepmd_data/')

{'atom_numbs': [54, 27], 'atom_names': ['H', 'O'], 'atom_types': array([1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]), 'orig': [0, 0, 0], 'cells': array([[[9.312845, 0.      , 0.      ],
        [0.      , 9.312845, 0.      ],
        [0.      , 0.      , 9.312845]],

       [[9.312845, 0.      , 0.      ],
        [0.      , 9.312845, 0.      ],
        [0.      , 0.      , 9.312845]],

       [[9.312845, 0.      , 0.      ],
        [0.      , 9.312845, 0.      ],
        [0.      , 0.      , 9.312845]],

       ...,

       [[9.312845, 0.      , 0.      ],
        [0.      , 9.312845, 0.      ],
        [0.      , 0.      , 9.312845]],

       [[9.312845, 0.      , 0.      ],
        [0.      , 9.312845, 0.      ],
        [0.      , 0.      , 9.312845]],

       [[9.

MultiSystems (1 systems containing 121 frames)